In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('data/raw/base_maturidade_final.csv', usecols=lambda column: column != 'Unnamed: 0')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   clientes_ativos         10000 non-null  int64  
 1   faturamento_mensal      10000 non-null  int64  
 2   faturamento_acumulado   10000 non-null  int64  
 3   anos_empresa            10000 non-null  float64
 4   tipo_servico            10000 non-null  object 
 5   ticket_medio            10000 non-null  float64
 6   mix_receita             10000 non-null  object 
 7   canais_vendas           10000 non-null  int64  
 8   churn                   10000 non-null  float64
 9   parcerias               10000 non-null  object 
 10  investimento_externo    10000 non-null  object 
 11  branding_reputacao      10000 non-null  object 
 12  network_time            10000 non-null  object 
 13  uso_tecnologia          10000 non-null  object 
 14  roadmap_produto         10000 non-null 

In [25]:
# colunas categoricas
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
# colunas numericas
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()

In [26]:
for col in df.select_dtypes(include=['object']).columns:
    print(f'Coluna: {col}')
    print(df[col].unique())
    print()

Coluna: tipo_servico
['Serviço' 'Produto']

Coluna: mix_receita
['3–5 clientes >50%' 'Receita diversificada' '1–2 clientes >50%']

Coluna: parcerias
['Algumas' 'Poucas' 'Muitas' 'Nenhuma']

Coluna: investimento_externo
['Série A+' 'Aceleradora/Anjo' 'Não' 'Seed/Pré-Série A']

Coluna: branding_reputacao
['Reconhecimento forte' 'Não reconhecida' 'Algum reconhecimento']

Coluna: network_time
['Sim, forte' 'Ocasionalmente' 'Não']

Coluna: uso_tecnologia
['Alto' 'Médio' 'Baixo']

Coluna: roadmap_produto
['Parcial' 'Não existe' 'Claro e executado']

Coluna: eficiencia_operacional
['Parcialmente' 'Quase tudo' 'Quase nada']

Coluna: escala_tecnologica
['Parcialmente' 'Não' 'Sim']

Coluna: financas_unidade
['Algumas' 'Não' 'Sim, regularmente']

Coluna: experiencia_lideranca
['Experiência forte' 'Sem experiência' 'Experiência parcial']

Coluna: treinamento
['Às vezes' 'Regularmente' 'Nunca']

Coluna: governanca_financeira
['Não existem controles' 'DRE e auditoria' 'Controles básicos']



In [27]:
# divisão treino, teste e validaçao em csv diferentes 65/25/10 
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.35, random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=0.2857, random_state=42) # 0.2857 * 0.35 = 0.10
# salvando os datasets

# Separando features (X) e labels (y)
target_column = 'score_total'

x_train = train_df.drop(columns=[target_column])
y_train = train_df[target_column]

x_test = test_df.drop(columns=[target_column])
y_test = test_df[target_column]

x_validation = val_df.drop(columns=[target_column])
y_validation = val_df[target_column]

# Salvando os datasets em arquivos CSV
x_train.to_csv('data/raw/test_split/x_train.csv', index=False)
y_train.to_csv('data/raw/test_split/y_train.csv', index=False)
x_test.to_csv('data/raw/test_split/x_test.csv', index=False)
y_test.to_csv('data/raw/test_split/y_test.csv', index=False)
x_validation.to_csv('data/raw/test_split/x_validation.csv', index=False)
y_validation.to_csv('data/raw/test_split/y_validation.csv', index=False)

# Exibindo as formas dos datasets
print(f'x_train shape: {x_train.shape}, y_train shape: {y_train.shape}')
print(f'x_test shape: {x_test.shape}, y_test shape: {y_test.shape}')
print(f'x_validation shape: {x_validation.shape}, y_validation shape: {y_validation.shape}')


x_train shape: (6500, 24), y_train shape: (6500,)
x_test shape: (2500, 24), y_test shape: (2500,)
x_validation shape: (1000, 24), y_validation shape: (1000,)


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --- 1. Carregando os CSVs ---
x_train = pd.read_csv('data/raw/test_split/x_train.csv')
y_train = pd.read_csv('data/raw/test_split/y_train.csv')

x_test = pd.read_csv('data/raw/test_split/x_test.csv')
y_test = pd.read_csv('data/raw/test_split/y_test.csv')

x_validation = pd.read_csv('data/raw/test_split/x_validation.csv')
y_validation = pd.read_csv('data/raw/test_split/y_validation.csv')

# --- 2. Definindo colunas categóricas ---
col_one_hot = ['tipo_servico']
col_ordinal = [
    'mix_receita', 'parcerias', 'investimento_externo', 'branding_reputacao',
    'network_time', 'uso_tecnologia', 'roadmap_produto', 'eficiencia_operacional',
    'escala_tecnologica', 'financas_unidade', 'experiencia_lideranca',
    'treinamento', 'governanca_financeira'
]

# --- 3. Ordem das categorias para o OrdinalEncoder ---
ordinal_categories = [
    ['1–2 clientes >50%', '3–5 clientes >50%', 'Receita diversificada'],      
    ['Nenhuma', 'Poucas', 'Algumas', 'Muitas'],                               
    ['Não', 'Aceleradora/Anjo', 'Seed/Pré-Série A', 'Série A+'],              
    ['Não reconhecida', 'Algum reconhecimento', 'Reconhecimento forte'],      
    ['Não', 'Ocasionalmente', 'Sim, forte'],                                  
    ['Baixo', 'Médio', 'Alto'],                                               
    ['Não existe', 'Parcial', 'Claro e executado'],                           
    ['Quase nada', 'Parcialmente', 'Quase tudo'],                             
    ['Não', 'Parcialmente', 'Sim'],                                           
    ['Não', 'Algumas', 'Sim, regularmente'],                                  
    ['Sem experiência', 'Experiência parcial', 'Experiência forte'],          
    ['Nunca', 'Às vezes', 'Regularmente'],                                    
    ['Não existem controles', 'Controles básicos', 'DRE e auditoria']         
]

# --- 4. Criando encoders ---
one_hot_encoder = OneHotEncoder(drop='first', handle_unknown='ignore')
ordinal_encoder = OrdinalEncoder(categories=ordinal_categories)

# --- 5. Criando pré-processador com normalização ---
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', one_hot_encoder, col_one_hot),
        ('ordinal', ordinal_encoder, col_ordinal),
        ('scaler', StandardScaler(), list(set(x_train.columns) - set(col_one_hot) - set(col_ordinal)))
    ],
    remainder='drop'
)

# --- 6. Criar pipeline final ---
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# --- 7. Fit no treino e transform nos outros ---
X_train_transformed = pipeline.fit_transform(x_train)
X_test_transformed = pipeline.transform(x_test)
X_val_transformed = pipeline.transform(x_validation)

# --- 8. Converter para DataFrame e salvar em CSV ---
# Pegar nomes das colunas geradas pelo OneHotEncoder e OrdinalEncoder
one_hot_cols = pipeline.named_steps['preprocessor'].named_transformers_['onehot'].get_feature_names_out(col_one_hot)
ordinal_cols = col_ordinal
numeric_cols = list(set(x_train.columns) - set(col_one_hot) - set(col_ordinal))

processed_columns = list(one_hot_cols) + ordinal_cols + numeric_cols

# Criar DataFrames
X_train_df = pd.DataFrame(X_train_transformed, columns=processed_columns, index=x_train.index)
X_test_df = pd.DataFrame(X_test_transformed, columns=processed_columns, index=x_test.index)
X_val_df = pd.DataFrame(X_val_transformed, columns=processed_columns, index=x_validation.index)

# Salvar CSVs
X_train_df.to_csv('data/processed/x_train_encoded.csv', index=False)
X_test_df.to_csv('data/processed/x_test_encoded.csv', index=False)
X_val_df.to_csv('data/processed/x_validation_encoded.csv', index=False)